# MNIST

In [1]:
%reset


Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Dense, Reshape, BatchNormalization, Activation, Conv2D, Conv2DTranspose, LeakyReLU, Flatten
from lib import
import os
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [8]:
def plot_images(generator,
                noise_input,
                noise_label=None,
                noise_codes=None,
                show=False,
                step=0,
                model_name="gan"):
    """
    # Arguments
        generator (Model)
        noise_input (ndarray)
        show (bool)
        step (int)
        model_name (string)

    """
    os.makedirs(model_name, exist_ok=True)
    filename = os.path.join(model_name, "%05d.png" % step)
    rows = int(math.sqrt(noise_input.shape[0]))
    if noise_label is not None:
        noise_input = [noise_input, noise_label]
        if noise_codes is not None:
            noise_input += noise_codes

    images = generator.predict(noise_input)
    plt.figure(figsize=(2.2, 2.2))
    num_images = images.shape[0]
    image_size = images.shape[1]
    for i in range(num_images):
        plt.subplot(rows, rows, i + 1)
        image = np.reshape(images[i], [image_size, image_size])
        plt.imshow(image, cmap='gray')
        plt.axis('off')
    plt.savefig(filename)
    if show:
        plt.show()
    else:
        plt.close('all')

In [3]:
def build_generator(inputs, image_size):
    # Stack: BN-ReLU-Conv2DTranspose
    # inputs: z-vector(noise)
    # image_size: Target size
    # return: Model
    
    image_resize = image_size // 4
    kernel_size = 5
    layer_filters = [128, 64, 32, 1]
    
    x = Dense(image_resize*image_resize*layer_filters[0])(inputs)
    x = Reshape((image_resize, image_resize, layer_filters[0]))(x)
    
    for filters in layer_filters:
        # 1st, 2nd Conv layers: strides = 2
        # 3rd, 4th Conv layers: strides = 1
        if filters > layer_filters[-2]: #128, 64
            strides = 2
        else:
            strides = 1
        # BN-ReLU-Conv2DTranspose
        x = BatchNormalization()(x)
        x = Conv2DTranspose(filters=filters,
                            kernel_size=kernel_size,
                            strides=strides,
                            padding='same'
                           )(x)
        
    x = Activation('sigmoid')(x)
    generator = keras.Model(inputs, x, name='generator')
    return generator

In [4]:
def build_discriminator(inputs):
    # BN으로는 수렴하지 않음
    # Stack: LeakyReLU-Conv2D
    # inputs: Image
    # return: Model
    
    kernel_size = 5
    layer_filters = [32, 64, 128, 256]
    
    x = inputs
    
    for filters in layer_filters:
        # 1st, 2nd, 3rd Conv layers: strides = 2
        # 4th Conv layers: strides = 1
        if filters == layer_filters[-1]:
            strides = 1
        else:
            strides = 2
        # LeakyReLU-Conv2D
        x = LeakyReLU(alpha=0.2)(x)
        x = Conv2D(filters=filters,
                   kernel_size=kernel_size,
                   strides=strides,
                   padding='same'
                  )(x)
    x = Flatten()(x)
    x = Dense(1)(x)
    x = Activation('sigmoid')(x)
    discriminator = keras.Model(inputs, x, name='discriminator')
    return discriminator

In [5]:
def build_and_train_models():
    (x_train, _), (_, _) = keras.datasets.mnist.load_data()
    
    # Reshape & Normalize
    image_size = x_train.shape[1]
    x_train = np.reshape(x_train, [-1, image_size, image_size, 1])
    x_train = x_train.astype('float32')/255
    
    model_name = 'dcgan_mnist'
    # Network Params
    latent_size = 100 # z-vector dimension
    batch_size = 64
    train_steps = 40000
    lr = 2e-4
    decay = 6e-8 # decays the lerning rate over time
    input_shape = (image_size, image_size, 1)
    
    # Discriminator model
    inputs = keras.Input(shape=input_shape, name='discriminator_input')
    discriminator = build_discriminator(inputs)
    optimizer = keras.optimizers.RMSprop(learning_rate=lr, decay=decay)
    discriminator.compile(loss='binary_crossentropy',
                         optimizer=optimizer,
                         metrics=['accuracy']
                        )
    discriminator.summary()
    
    # Generator model
    input_shape = (latent_size, )
    inputs = keras.Input(shape=input_shape, name='z_input')
    generator = build_generator(inputs, image_size)
    generator.summary()
    
    # Adversarial model
    optimizer = keras.optimizers.RMSprop(learning_rate=lr*0.5,
                                         decay=decay*0.5,
                                        )
    discriminator.trainable = False
    ## Adversarial = Generator + Discriminator
    adversarial = keras.Model(inputs, discriminator(generator(inputs)), name='adversarial')
    adversarial.compile(loss='binary_crossentropy',
                        optimizer=optimizer,
                        metrics=['accuracy']
                       )
    adversarial.summary()
    
    models = (generator, discriminator, adversarial)
    params = (batch_size, latent_size, train_steps, model_name)
    train(models, x_train, params)

In [6]:
def train(models, x_train, params):
    # Discriminator와 Adversarial Model을 배치 단위로 번갈아 훈련
    ## Discriminator는 제대로 레이블이 붙은 진짜와 가짜 이미지를 가지고 훈련
    ## Adversarial는 진짜인 척하는 가짜 이미지로 훈련
    
    # GAN Model
    generator, discriminator, adversarial = models
    
    # Network Params
    batch_size, latent_size, train_steps, model_name = params
    
    # Save Generator Images every 500 epochs
    save_interval = 500
    
    # Noise_Input
    noise_input = np.random.uniform(-1.0, 1.0, size=[16, latent_size]) # 16 x 100
    
    train_size = x_train.shape[0]
    for i in range(train_steps): # train_steps: 40,000
        # First - Discriminator Train
        # Real Images
        rand_indexes = np.random.randint(0, train_size, size=batch_size)
        real_images = x_train[rand_indexes]
        
        # Fake Images
        # Make noise by using Uniform Distribution
        noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])
        fake_images = generator.predict(noise)
        
        # x: Real Images + Fake Images
        x = np.concatenate((real_images, fake_images))
        # y: real_images-1, fake_images-0
        y = np.ones([2*batch_size, 1])
        y[batch_size:, :] = 0.0
        
        loss, acc = discriminator.train_on_batch(x, y)
        log = "%d: [discriminator loss: %f, acc: %f]" %(i, loss, acc)
        
        # Second - Adversarial Train
        noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])
        y = np.ones([batch_size, 1])
        
        loss, acc = adversarial.train_on_batch(noise, y)
        log = "%s: [adversarial loss: %f, acc: %f]" %(log, loss, acc)
        print(log)
        
        # Show generator images per 500 epochs
        if (i+1) % save_interval == 0: # 500
            if (i+1) == train_steps: # 40,000
                show = True
            else:
                show = False
            
            plot_images(generator,
                        noise_input=noise_input,
                        show=show,
                        step=(i+1),
                        model_name=model_name
                       )
            
        generator.save(model_name + ".h5")

In [9]:
if __name__=="__main__":
    build_and_train_models()

Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 discriminator_input (InputL  [(None, 28, 28, 1)]      0         
 ayer)                                                           
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 28, 28, 1)         0         
                                                                 
 conv2d_4 (Conv2D)           (None, 14, 14, 32)        832       
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 14, 14, 32)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 7, 7, 64)          51264     
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 7, 7, 64)          0         
                                                     

2023-02-18 15:39:54.665287: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-18 15:39:55.047639: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-18 15:39:55.663090: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


0: [discriminator loss: 0.691425, acc: 0.523438]: [adversarial loss: 0.951768, acc: 0.000000]
1: [discriminator loss: 0.598164, acc: 0.531250]: [adversarial loss: 1.412784, acc: 0.000000]
2: [discriminator loss: 0.481725, acc: 0.765625]: [adversarial loss: 1.377934, acc: 0.000000]
3: [discriminator loss: 0.353327, acc: 1.000000]: [adversarial loss: 2.092016, acc: 0.000000]
4: [discriminator loss: 0.213590, acc: 1.000000]: [adversarial loss: 0.461585, acc: 0.937500]
5: [discriminator loss: 0.255455, acc: 1.000000]: [adversarial loss: 4.538691, acc: 0.000000]
6: [discriminator loss: 0.215166, acc: 0.953125]: [adversarial loss: 2.128721, acc: 0.000000]
7: [discriminator loss: 0.080942, acc: 1.000000]: [adversarial loss: 1.755581, acc: 0.000000]
8: [discriminator loss: 0.055702, acc: 1.000000]: [adversarial loss: 1.617808, acc: 0.015625]
9: [discriminator loss: 0.044253, acc: 1.000000]: [adversarial loss: 1.579135, acc: 0.078125]
10: [discriminator loss: 0.035707, acc: 1.000000]: [adversar

31: [discriminator loss: 0.002482, acc: 1.000000]: [adversarial loss: 0.063309, acc: 1.000000]
32: [discriminator loss: 0.001690, acc: 1.000000]: [adversarial loss: 0.057718, acc: 1.000000]
33: [discriminator loss: 0.001593, acc: 1.000000]: [adversarial loss: 0.050352, acc: 1.000000]
34: [discriminator loss: 0.001408, acc: 1.000000]: [adversarial loss: 0.041533, acc: 1.000000]
35: [discriminator loss: 0.001617, acc: 1.000000]: [adversarial loss: 0.029212, acc: 1.000000]
36: [discriminator loss: 0.001380, acc: 1.000000]: [adversarial loss: 0.028257, acc: 1.000000]
37: [discriminator loss: 0.001136, acc: 1.000000]: [adversarial loss: 0.026108, acc: 1.000000]
38: [discriminator loss: 0.000953, acc: 1.000000]: [adversarial loss: 0.020945, acc: 1.000000]
39: [discriminator loss: 0.000875, acc: 1.000000]: [adversarial loss: 0.021184, acc: 1.000000]
40: [discriminator loss: 0.000756, acc: 1.000000]: [adversarial loss: 0.016826, acc: 1.000000]
41: [discriminator loss: 0.000838, acc: 1.000000]:

62: [discriminator loss: 0.000159, acc: 1.000000]: [adversarial loss: 0.001604, acc: 1.000000]
63: [discriminator loss: 0.000123, acc: 1.000000]: [adversarial loss: 0.002286, acc: 1.000000]
64: [discriminator loss: 0.000118, acc: 1.000000]: [adversarial loss: 0.002928, acc: 1.000000]
65: [discriminator loss: 0.000161, acc: 1.000000]: [adversarial loss: 0.002066, acc: 1.000000]
66: [discriminator loss: 0.000119, acc: 1.000000]: [adversarial loss: 0.001754, acc: 1.000000]
67: [discriminator loss: 0.000141, acc: 1.000000]: [adversarial loss: 0.001129, acc: 1.000000]
68: [discriminator loss: 0.000115, acc: 1.000000]: [adversarial loss: 0.001598, acc: 1.000000]
69: [discriminator loss: 0.000077, acc: 1.000000]: [adversarial loss: 0.002190, acc: 1.000000]
70: [discriminator loss: 0.000104, acc: 1.000000]: [adversarial loss: 0.000799, acc: 1.000000]
71: [discriminator loss: 0.000079, acc: 1.000000]: [adversarial loss: 0.001191, acc: 1.000000]
72: [discriminator loss: 0.000073, acc: 1.000000]:

...

403: [discriminator loss: 0.035473, acc: 0.992188]: [adversarial loss: 1.892673, acc: 0.156250]
404: [discriminator loss: 0.057823, acc: 0.992188]: [adversarial loss: 8.080188, acc: 0.000000]
405: [discriminator loss: 0.200335, acc: 0.937500]: [adversarial loss: 0.067271, acc: 0.984375]
406: [discriminator loss: 0.950042, acc: 0.609375]: [adversarial loss: 14.567698, acc: 0.000000]
407: [discriminator loss: 1.040662, acc: 0.750000]: [adversarial loss: 4.618074, acc: 0.093750]
408: [discriminator loss: 0.049434, acc: 0.992188]: [adversarial loss: 3.858637, acc: 0.062500]
409: [discriminator loss: 0.035517, acc: 0.984375]: [adversarial loss: 4.223283, acc: 0.046875]
410: [discriminator loss: 0.083847, acc: 0.976562]: [adversarial loss: 3.503093, acc: 0.015625]
411: [discriminator loss: 0.036358, acc: 0.984375]: [adversarial loss: 3.679577, acc: 0.062500]
412: [discriminator loss: 0.068042, acc: 0.976562]: [adversarial loss: 3.932732, acc: 0.000000]
413: [discriminator loss: 0.032730, acc

434: [discriminator loss: 0.070958, acc: 0.960938]: [adversarial loss: 3.167054, acc: 0.125000]
435: [discriminator loss: 0.075434, acc: 0.984375]: [adversarial loss: 6.113436, acc: 0.000000]
436: [discriminator loss: 0.200067, acc: 0.960938]: [adversarial loss: 1.257473, acc: 0.343750]
437: [discriminator loss: 0.261624, acc: 0.875000]: [adversarial loss: 8.713657, acc: 0.015625]
438: [discriminator loss: 0.648359, acc: 0.820312]: [adversarial loss: 0.630928, acc: 0.562500]
439: [discriminator loss: 0.413416, acc: 0.781250]: [adversarial loss: 8.656566, acc: 0.000000]
440: [discriminator loss: 0.249482, acc: 0.898438]: [adversarial loss: 4.396397, acc: 0.031250]
441: [discriminator loss: 0.158906, acc: 0.929688]: [adversarial loss: 5.341274, acc: 0.062500]
442: [discriminator loss: 0.050807, acc: 0.992188]: [adversarial loss: 4.708757, acc: 0.062500]
443: [discriminator loss: 0.106816, acc: 0.960938]: [adversarial loss: 3.302134, acc: 0.078125]
444: [discriminator loss: 0.145021, acc:

465: [discriminator loss: 0.088282, acc: 0.968750]: [adversarial loss: 2.922676, acc: 0.125000]
466: [discriminator loss: 0.168670, acc: 0.945312]: [adversarial loss: 8.038531, acc: 0.000000]
467: [discriminator loss: 0.410174, acc: 0.890625]: [adversarial loss: 1.231822, acc: 0.375000]
468: [discriminator loss: 0.553173, acc: 0.765625]: [adversarial loss: 9.690373, acc: 0.000000]
469: [discriminator loss: 1.026799, acc: 0.687500]: [adversarial loss: 1.761951, acc: 0.250000]
470: [discriminator loss: 0.316224, acc: 0.875000]: [adversarial loss: 5.088668, acc: 0.062500]
471: [discriminator loss: 0.268272, acc: 0.906250]: [adversarial loss: 3.619834, acc: 0.078125]
472: [discriminator loss: 0.238177, acc: 0.906250]: [adversarial loss: 4.448476, acc: 0.046875]
473: [discriminator loss: 0.236096, acc: 0.898438]: [adversarial loss: 4.445969, acc: 0.062500]
474: [discriminator loss: 0.248524, acc: 0.898438]: [adversarial loss: 3.759269, acc: 0.140625]
475: [discriminator loss: 0.213269, acc:

496: [discriminator loss: 0.126830, acc: 0.968750]: [adversarial loss: 3.295642, acc: 0.062500]
497: [discriminator loss: 0.241912, acc: 0.906250]: [adversarial loss: 2.934793, acc: 0.046875]
498: [discriminator loss: 0.348723, acc: 0.882812]: [adversarial loss: 3.806605, acc: 0.031250]
499: [discriminator loss: 0.332623, acc: 0.875000]: [adversarial loss: 2.493750, acc: 0.187500]


NameError: name 'os' is not defined